# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
city_weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lensk,60.73,114.93,74.48,53,100,2.08,RU,1593607131
1,1,jamestown,42.10,-79.24,69.01,88,75,6.93,US,1593606990
2,2,omboue,-1.57,9.26,77.65,73,100,10.65,GA,1593607131
3,3,pyapon,16.28,95.68,80.02,84,100,11.86,MM,1593607131
4,4,port alfred,-33.59,26.89,80.01,28,0,5.99,ZA,1593607132


In [14]:
city_weather_df.count()

City_ID       565
City          565
Lat           565
Lng           565
Max Temp      565
Humidity      565
Cloudiness    565
Wind Speed    565
Country       562
Date          565
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
humidity = city_weather_df["Humidity"]
maxhumidity = humidity.max()
#Use the Lat and Lng as location
locations = city_weather_df[["Lat", "Lng"]]
#Humidity as the weight
weight = city_weather_df["Humidity"]

In [59]:
#Plot heat maps
figure = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight,dissipating=False, max_intensity=maxhumidity,point_radius=3)
figure.add_layer(heat_layer)

# Display figure



In [62]:
#Plot heatmap
fig = gmaps.figure(center=(45.0, -0.5), zoom_level=1.5)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=1)

fig.add_layer(heat_layer)




### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [111]:
ideal_weather = city_weather_df[(city_weather_df['Max Temp'].between(70,80)) & (city_weather_df["Cloudiness"] == 0 & (city_weather_df['Wind Speed']<10) & (city_weather_df['Humidity']<=45)]
ideal_weather.reset_index(inplace=True)
city_data.to_csv(output_data_file, index=False)
ideal_weather


SyntaxError: invalid syntax (<ipython-input-111-f18d5530aaed>, line 1)

In [ ]:
new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

In [106]:
#clean up DataSet by remooving NAN
clean_weather = ideal_weather.dropna()
clean_weather.drop('index', axis=1, inplace=True)
clean_weather



C:\Users\Posh_2\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,4,port alfred,-33.59,26.89,80.01,28,0,5.99,ZA,1593607132
1,21,fuxin,42.02,121.66,77.54,42,27,5.01,CN,1593607134
2,29,mavrovi anovi,41.70,20.76,86.00,40,40,3.36,MK,1593607136
4,85,east london,-33.02,27.91,73.40,43,0,4.70,ZA,1593607146
5,198,zhangye,38.93,100.45,81.12,15,0,7.96,CN,1593607144
6,236,asau,46.43,26.40,84.20,42,20,4.70,RO,1593607170
7,248,hwange,-18.37,26.50,79.07,27,0,5.77,ZW,1593607172
8,254,korla,41.76,86.15,87.64,12,100,7.00,CN,1593606898
9,295,baykit,61.67,96.37,76.66,37,63,3.56,RU,1593607180
10,348,porangatu,-13.44,-49.15,86.13,36,2,2.37,BR,1593607188


In [ ]:
ideal_weather.reset_index(inplace=True)

In [107]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

NameError: name 'smaller_city_df' is not defined

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = []

for x in range(len(hotel_df)):
    lat = hotel_df.loc[x]['Latitude']
    lng = hotel_df.loc[x]['Longitude']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    
    try:
        hotel_df.append(response['results'][0]['name'])
    except:
        hotel_df.append("")      

In [ ]:
hotel_df["Hotel Name"] = hotel_df
hotel_df
hotel_df = hotel_df.dropna(how='any')
hotel_df

In [ ]:
 #Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
figure.add_layer(markers)

# Display figure
figure

In [9]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

NameError: name 'weather_data' is not defined

In [10]:
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Clouds"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

NameError: name 'city_weather_df' is not defined

In [ ]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")